In [1]:
import paddle.fluid as fluid
from paddle.fluid.param_attr import ParamAttr
import paddle
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib; matplotlib.use('Qt5Agg') 

In [53]:
BUF_SIZE=300
BATCH_SIZE=170

#用于训练的数据提供器，每次从缓存中随机读取批次大小的数据
train_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.uci_housing.train(), 
                          buf_size=BUF_SIZE),                    
    batch_size=BATCH_SIZE)   

#用于测试的数据提供器，每次从缓存中随机读取批次大小的数据
test_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.uci_housing.test(),
                          buf_size=BUF_SIZE),
    batch_size=BATCH_SIZE)  

In [3]:
#用于打印，查看uci_housing数据
train_data=paddle.dataset.uci_housing.train();
sampledata=next(train_data())
print(sampledata)

(array([-0.0405441 ,  0.06636364, -0.32356227, -0.06916996, -0.03435197,
        0.05563625, -0.03475696,  0.02682186, -0.37171335, -0.21419304,
       -0.33569506,  0.10143217, -0.21172912]), array([24.]))


In [4]:
#定义张量变量x，表示13维的特征值
x = fluid.layers.data(name='x', shape=[13], dtype='float32')
y = fluid.layers.data(name='y', shape=[1], dtype='float32')

param_attr = ParamAttr(name='fc_1.w_0')
bias_attr = ParamAttr(name='fc_1.b_0')

#定义张量y,表示目标值


#定义一个简单的线性网络,连接输入和输出的全连接层
#input:输入tensor;
#size:该层输出单元的数目
#act:激活函数
y_predict = fluid.layers.fc(input=x,size=1,act=None, param_attr=param_attr, bias_attr=bias_attr)

In [5]:
cost = fluid.layers.square_error_cost(input=y_predict, label=y) #求一个batch的损失值
avg_cost = fluid.layers.mean(cost)                              #对损失值求平均值

In [6]:
optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
opts = optimizer.minimize(avg_cost)

In [7]:
test_program = fluid.default_main_program().clone(for_test=True)

In [8]:
use_cuda = False                         #use_cuda为False,表示运算场所为CPU;use_cuda为True,表示运算场所为GPU           
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
exe = fluid.Executor(place)              #创建一个Executor实例exe
exe.run(fluid.default_startup_program()) #Executor的run()方法执行startup_program(),进行参数初始化

[]

In [9]:
# 定义输入数据维度
feeder = fluid.DataFeeder(place=place, feed_list=[x, y])#feed_list:向模型输入的变量表或变量表名

In [10]:
iter=0;
iters=[]
train_costs=[]

def draw_train_process(iters,train_costs):
    title="training cost"
    plt.title(title, fontsize=24)
    plt.xlabel("iter", fontsize=14)
    plt.ylabel("cost", fontsize=14)
    plt.plot(iters, train_costs,color='red',label='training cost') 
    plt.grid()
    plt.show()

In [11]:
EPOCH_NUM=60
model_save_dir = "./boston_model"

for pass_id in range(EPOCH_NUM):                                  #训练EPOCH_NUM轮
    # 开始训练并输出最后一个batch的损失值
    train_cost = 0
    for batch_id, data in enumerate(train_reader()):              #遍历train_reader迭代器
        train_cost = exe.run(program=fluid.default_main_program(),#运行主程序
                             feed=feeder.feed(data),              #喂入一个batch的训练数据，根据feed_list和data提供的信息，将输入数据转成一种特殊的数据结构
                             fetch_list=[avg_cost])    
        if batch_id % 40 == 0:
            print("Pass:%d, Cost:%0.5f" % (pass_id, train_cost[0][0]))    #打印最后一个batch的损失值
        iter=iter+BATCH_SIZE
        iters.append(iter)
        train_costs.append(train_cost[0][0])
       
   
    # 开始测试并输出最后一个batch的损失值
    test_cost = 0
    for batch_id, data in enumerate(test_reader()):               #遍历test_reader迭代器
        test_cost= exe.run(program=test_program, #运行测试cheng
                            feed=feeder.feed(data),               #喂入一个batch的测试数据
                            fetch_list=[avg_cost])                #fetch均方误差
    print('Test:%d, Cost:%0.5f' % (pass_id, test_cost[0][0]))     #打印最后一个batch的损失值
    
    #保存模型
    # 如果保存路径不存在就创建
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)
print ('save models to %s' % (model_save_dir))
#保存训练参数到指定路径中，构建一个专门用预测的program
fluid.io.save_inference_model(model_save_dir,   #保存推理model的路径
                                  ['x'],            #推理（inference）需要 feed 的数据
                                  [y_predict],      #保存推理（inference）结果的 Variables
                                  exe)              #exe 保存 inference model
# fluid.io.save_params(executor=exe,model_save_dir)
draw_train_process(iters,train_costs)

# get tensor
place = fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())
ret = fluid.global_scope().find_var("fc_1.w_0").get_tensor()
ret1 = fluid.global_scope().find_var("fc_1.b_0").get_tensor()

ret = np.array(ret)
ret1 = np.array(ret1)
print (ret.shape)
print(ret)
print (ret1.shape)
print(ret1)

Pass:0, Cost:683.23425
Test:0, Cost:269.62338
Pass:1, Cost:625.81671
Test:1, Cost:237.23433
Pass:2, Cost:553.62592
Test:2, Cost:209.23167
Pass:3, Cost:531.76160
Test:3, Cost:184.05058
Pass:4, Cost:459.14618
Test:4, Cost:162.52959
Pass:5, Cost:409.44305
Test:5, Cost:143.50392
Pass:6, Cost:379.66238
Test:6, Cost:126.87182
Pass:7, Cost:370.35587
Test:7, Cost:112.50027
Pass:8, Cost:329.32227
Test:8, Cost:99.50972
Pass:9, Cost:284.17303
Test:9, Cost:88.12251
Pass:10, Cost:226.84444
Test:10, Cost:78.10293
Pass:11, Cost:235.54405
Test:11, Cost:70.06310
Pass:12, Cost:210.18846
Test:12, Cost:62.40219
Pass:13, Cost:225.26248
Test:13, Cost:55.65991
Pass:14, Cost:200.62590
Test:14, Cost:50.22062
Pass:15, Cost:188.08066
Test:15, Cost:45.47271
Pass:16, Cost:158.25119
Test:16, Cost:41.31053
Pass:17, Cost:135.56308
Test:17, Cost:37.60035
Pass:18, Cost:129.20490
Test:18, Cost:34.54728
Pass:19, Cost:130.73126
Test:19, Cost:31.68417
Pass:20, Cost:104.45837
Test:20, Cost:29.37806
Pass:21, Cost:126.93477
T

E:\Anaconda\envs\env_paddle\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  if sys.path[0] == '':


In [46]:
test1 = np.array([0.42616306, -0.11363636,  0.25525005, -0.06916996,  0.28457807,
       -0.14440207,  0.17327599, -0.19893267,  0.62828665,  0.49191383,
        0.18558153, -0.0686218 ,  0.40637243])
test2 = np.array([1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ])
test3 = []

test3.append(test1)

print(type(test3))
ress = test1.dot(ret)
ress

<class 'list'>


array([0.20899439])

In [47]:
[inference_program,                             #推理的program
     feed_target_names,                             #需要在推理program中提供数据的变量名称
     fetch_targets] = fluid.io.load_inference_model(#fetch_targets: 推断结果
                                    model_save_dir, #model_save_dir:模型训练路径 
                                    infer_exe)      #infer_exe: 预测用executor

test_11 = np.array(test3).astype("float32")

results = infer_exe.run(inference_program,                              #预测模型
                            feed={feed_target_names[0]: np.array(test_11)},  #喂入要预测的x值
                            fetch_list=fetch_targets)                       #得到推测结果 
results

[array([[14.668234]], dtype=float32)]

## 预测

In [14]:
infer_exe = fluid.Executor(place)    #创建推测用的executor
inference_scope = fluid.core.Scope() #Scope指定作用域

In [15]:
infer_results=[]
groud_truths=[]

#绘制真实值和预测值对比图
def draw_infer_result(groud_truths,infer_results):
    title='Boston'
    plt.title(title, fontsize=24)
    x = np.arange(1,20) 
    y = x
    plt.plot(x, y)
    plt.xlabel('ground truth', fontsize=14)
    plt.ylabel('infer result', fontsize=14)
    plt.scatter(groud_truths, infer_results,color='green',label='training cost') 
    plt.grid()
    plt.show()

In [49]:
with fluid.scope_guard(inference_scope):#修改全局/默认作用域（scope）, 运行时中的所有变量都将分配给新的scope。
    #从指定目录中加载 推理model(inference model)
    [inference_program,                             #推理的program
     feed_target_names,                             #需要在推理program中提供数据的变量名称
     fetch_targets] = fluid.io.load_inference_model(#fetch_targets: 推断结果
                                    model_save_dir, #model_save_dir:模型训练路径 
                                    infer_exe)      #infer_exe: 预测用executor
    #获取预测数据
    infer_reader = paddle.batch(paddle.dataset.uci_housing.test(),  #获取uci_housing的测试数据
                          batch_size=200)                           #从测试数据中读取一个大小为200的batch数据
    #从test_reader中分割x
    test_data = next(infer_reader())
    print(test_data)
    test_x = np.array([data[0] for data in test_data]).astype("float32")
    test_y= np.array([data[1] for data in test_data]).astype("float32")
    results = infer_exe.run(inference_program,                              #预测模型
                            feed={feed_target_names[0]: np.array(test_x)},  #喂入要预测的x值
                            fetch_list=fetch_targets)                       #得到推测结果 
                            
    print("infer results: (House Price)")
    for idx, val in enumerate(results[0]):
        print("%d: %.2f" % (idx, val))
        infer_results.append(val)
    print("ground truth:")
    for idx, val in enumerate(test_y):
        print("%d: %.2f" % (idx, val))
        groud_truths.append(val)
    draw_infer_result(groud_truths,infer_results)

[(array([ 0.42616306, -0.11363636,  0.25525005, -0.06916996,  0.28457807,
       -0.14440207,  0.17327599, -0.19893267,  0.62828665,  0.49191383,
        0.18558153, -0.0686218 ,  0.40637243]), array([8.5])), (array([ 0.72279828, -0.11363636,  0.25525005, -0.06916996,  0.28457807,
       -0.11527771,  0.32363645, -0.21548279,  0.62828665,  0.49191383,
        0.18558153,  0.07134996,  0.28495962]), array([5.])), (array([ 0.19222996, -0.11363636,  0.25525005, -0.06916996,  0.21461922,
       -0.41131144,  0.32363645, -0.23797095,  0.62828665,  0.49191383,
        0.18558153,  0.03415696,  0.2948934 ]), array([11.9])), (array([ 0.09371235, -0.11363636,  0.25525005, -0.06916996,  0.21461922,
       -0.12964828,  0.32363645, -0.22823184,  0.62828665,  0.49191383,
        0.18558153, -0.06199009, -0.01443331]), array([27.9])), (array([ 0.0426028 , -0.11363636,  0.25525005, -0.06916996,  0.0870472 ,
       -0.12792381,  0.30200926, -0.2128184 ,  0.62828665,  0.49191383,
        0.18558153, -

E:\Anaconda\envs\env_paddle\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  from ipykernel import kernelapp as app
